### Project - Airline AI Assistant

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"

openai = OpenAI()
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [ ]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence. 
Always be accurate. If you do not know the answer, say so.
"""

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = ollama.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat).launch()

In [ ]:
ticket_prices = {
    "trivandrum": "$1999",
    "halifax": "$2102",
    "london": "$1200",
    "wellington": "$3022",
    "valencia": "$2032",
    "tokyo": "$2031",
    "new york": "$1222",
}

def get_ticket_price(destination_city):
    print(f"getting the ticket price for desitination city : {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "unknown destination. we do not operate to this location")
    return f"The price of a ticket to {destination_city} is {price}"

In [ ]:
get_ticket_price("abc")

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }
        }
    },
    "required": ["destination_city"],
    "additionalProperties": False
}

In [67]:
tools = [{"type": "function", "function": price_function}]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}}},
   'required': ['destination_city'],
   'additionalProperties': False}}]

In [ ]:
def chat_with_tool(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages, tools=tools)
    if response.choices[0].finish_reason == "tool_calls":
        tool_message = response.choices[0].message
        tools_response = handle_tools_call(tool_message)
        messages.append(tool_message)
        messages.append(tools_response)

        for message in messages:
            print(message)

        response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
        
    return response.choices[0].message.content

In [ ]:
def handle_tools_call(message):
   tools_call = message.tool_calls[0]
   if tools_call.function.name == "get_ticket_price":
      arguments = json.loads(tools_call.function.arguments)
      city = arguments.get("destination_city")
      price_details = get_ticket_price(city)
      response = {
         "role": "tool",
         "content": price_details,
         "tool_call_id": tools_call.id
      }
   return response

In [ ]:
gr.ChatInterface(fn=chat_with_tool).launch(inbrowser=True)